In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [2]:
data=pd.read_excel('fake_data.xlsx')
data

,Vendor,Date when delivery is needed,date of delivery,date of ordering
0,1,2018-12-21,2018-12-19,2018-12-01
1,2,2019-01-13,2019-01-23,2019-01-03
2,1,2018-10-12,2018-10-08,2018-09-20
3,1,2018-11-11,2018-10-29,2018-10-11
4,1,2018-11-01,2018-10-19,2018-10-01
5,3,2018-04-06,2018-03-30,2018-03-16
6,3,2018-05-20,2018-05-17,2018-05-02
7,1,2018-03-30,2018-03-22,2018-03-04
8,1,2018-04-13,2018-04-12,2018-03-25
9,1,2018-05-22,2018-05-19,2018-05-01


In [3]:
train_DN=data["date of delivery"] #stores the day on which delivery is needed to be dome
train_DN=[round(a.day+30.4*(a.month-1)) for a in train_DN] #convert it to the approximate day number of the year
train_ADP=data['date of delivery']-data['date of ordering']#Appropriate days prior: the number of days prior to Delivery needed date that the order should have been placed
train_ADP=[float(a.days) for a in train_ADP] # convert to float format

# First assuming independent of vendor. Model to predict how many days prior to need should we order.

In [4]:
num_epoch=15 # number of times we want to optimize

Here we build the model

Assuming it follows time series nature cyclic over a year

In [5]:
tf.reset_default_graph()
sess=tf.InteractiveSession()
#varible coeffitients and input/output defined here
x=tf.placeholder(tf.float32,shape=([None])) #input for day delivered
z=tf.placeholder(tf.float32,shape=([None])) #input for day of placing order

#variables to find by optimization
a_s0=tf.Variable(tf.random_normal([1]))
a_c0=tf.Variable(tf.random_normal([1]))
aS0=tf.Variable(5*tf.random_normal([1]))
aC0=tf.Variable(5*tf.random_normal([1]))
a_s1=tf.Variable(tf.random_normal([1]))
a_c1=tf.Variable(tf.random_normal([1]))
aS1=tf.Variable(5*tf.random_normal([1]))
aC1=tf.Variable(5*tf.random_normal([1]))
m=tf.Variable(tf.random_normal([1]))
c=tf.Variable(tf.random_normal([1]))

# expected output for days prior for placing order
Z=c+aS0*tf.sin(a_s0*x)+aC0*tf.cos(a_c0*x)+aS1*tf.sin(a_s1*x)+aC1*tf.cos(a_c1*x) 

#loss here is not regularized but should be to avoid overfit but who gives a fuck anyways
loss=tf.reduce_mean(tf.square(Z-z))
optimizer=tf.train.GradientDescentOptimizer(0.6)
train=optimizer.minimize(loss)
saver = tf.train.Saver()

Next we run the training session

In [6]:
sess.run(tf.global_variables_initializer())
for epochs in range(num_epoch):
    sess.run(train,feed_dict={x:train_DN,z:train_ADP})
    print(sess.run(loss,feed_dict={x:train_DN,z:train_ADP}),end=",   ")

save_path = saver.save(sess, os.getcwd()+"/tmp/model.ckpt")
print("\n test variable : %s" % c.eval())
print("\n Model saved in path: %s" % save_path)
print("\n optimization finished \n \n model variables saved")



36.6037,   48.1185,   11.1071,   9.63764,   5.21289,   3.40445,   4.19404,   2.95459,   3.14267,   3.23316,   3.25285,   3.05503,   3.15815,   2.03019,   4.83702,   
 test variable : [ 17.92088318]

 Model saved in path: C:\Users\mayank.bijay\Desktop\inventory/tmp/model.ckpt

 optimization finished 
 
 model variables saved


---------------------------------------------------------
test here: we load the saved model here and evaluate the how many days earlier we should order according to our model against check_date

In [7]:
saver.restore(sess, os.getcwd()+"/tmp/model.ckpt")
print("Model restored.")
check_date=[8,1]#=[8th jan]::[d,m] here we enter the date we need delivery to check how many days prior we should order
daysPrior=Z.eval(feed_dict={x:[check_date[0]+30.4*check_date[1]]})
print("you should order %s earlier to your required date of delivery" % int(round(daysPrior[0])))

INFO:tensorflow:Restoring parameters from C:\Users\mayank.bijay\Desktop\inventory/tmp/model.ckpt
Model restored.
you should order 18 earlier to your required date of delivery


In [8]:
daysPriorAll=Z.eval(feed_dict={x:train_DN})
diff=train_ADP-daysPriorAll
diff=np.linalg.norm(diff)
print("total squared difference: %s"%diff)

total squared difference: 7.61868018573
